## Automated feature Engineering algorithms:
- **Deep feature synthesis (DFS)**: DFS is a method for automated feature engineering that involves defining a set of "primitives" that represent basic transformations of the data, and then using these primitives to iteratively apply transformations and create new features. DFS can be used with a variety of tools, including Featuretools and Featuretools for Pandas.

- **Genetic programming**: Genetic programming is a method for automated feature engineering that uses a population of "individuals" that represent different combinations of features, and then applies a set of evolutionary operators (such as crossover, mutation, and selection) to search for the best combination of features for a given dataset. Genetic programming can be used with tools like TPOT and Feature-based Genetic Programming (FBGP).

- **Statistical tests and machine learning algorithms**: Some automated feature engineering tools use statistical tests or machine learning algorithms to identify important features and create new features from the data. Examples include autofeat, mfeat, Feature Fusion, and Feature-select.

- **Simple transformations and aggregations**: Many automated feature engineering tools allow users to create new features using simple transformations and aggregations of the data, such as scaling, normalization, and binning. Examples include DataRobot, auto-sklearn, and Feature-engine.

## Colab Settings

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Libraries:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image

---

## Presets:

In [ ]:
#Options
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)

---

## Data:

In [3]:
application = pd.read_csv("application_train.csv")
bureau = pd.read_csv("bureau.csv")
bureau_bal = pd.read_csv("bureau_balance.csv")
cc_bal = pd.read_csv("credit_card_balance.csv")
inst_payments = pd.read_csv("installments_payments.csv")
posc_bal = pd.read_csv("POS_CASH_balance.csv")
prev_application = pd.read_csv("previous_application.csv")

In [14]:
application.to_parquet("application.parquet")
bureau.to_parquet("bureau.parquet")
bureau_bal.to_parquet("bureau_balance.parquet")
prev_application.to_parquet("previous_application.parquet")
cc_bal.to_parquet("credit_card_balance.parquet")
inst_payments.to_parquet("installments_payments.parquet")
posc_bal.to_parquet("POS_CASH_balance.parquet")

---

## Functions:

### 1. Univariate Analysis

In [ ]:
def univariate_analysis(df):
    """
    Analyzes the columns in a Pandas DataFrame and returns a new DataFrame with the following information:
        - column name
        - column type (numeric or categorical)
        - percent missing
        - number of unique values
        - standard deviation (for numeric columns only)

    The resulting DataFrame is sorted by column type (categorical columns first) and then by percent missing (descending).

    Parameters:
        df (pandas.DataFrame): The input DataFrame.

    Returns:
        pandas.DataFrame: The resulting DataFrame with the analyzed information.
    """
    # Create an empty list to store the results for each column
    results = []

    # Iterate over the columns in the DataFrame
    for col in df.columns:
        # Get the number of unique values
        unique = df[col].nunique()
        # Calculate the percentage of missing values
        percent_missing = (df[col].isnull().sum() * 100 / len(df))
        # Determine if the column is numeric or categorical
        if df[col].dtype in ['int64', 'float64']:
            # Calculate the standard deviation for numeric columns
            std = df[col].std()
            std_formatted = '{:.4f}'.format(std)
            col_type = 'numeric'
        else:
            # Set the standard deviation to None for non-numeric columns
            std_formatted = None
            col_type = 'categorical'
        # Add the results for this column to the list
        results.append([col, col_type, percent_missing, unique, std_formatted])

    # Convert the list to a DataFrame
    result_df = pd.DataFrame(results, columns=['column_name', 'var_type', 'percent_missing', 'num_unique', 'std'])
    # Sort the DataFrame by var_type and then by percent_missing
    result_df_sorted = result_df.sort_values(by=['var_type', 'percent_missing'], ascending=False)
    # Reset the index of the sorted DataFrame, dropping the old index
    result_df_sort_indexed = result_df_sorted.reset_index(drop=True)

    return result_df_sort_indexed

### 2. Outlier Detection

## EDA

### application_train.csv
- Static data for all applications.
- One row represents one loan in our data sample.
- **SK_ID_CURR** is single loan ID that the person currently has.
- Static Data means the data the doesn't change with time or the data the doesn't change rapidly

In [ ]:
application.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [ ]:
univariate_analysis(application)

,column_name,var_type,percent_missing,num_unique,std
0,COMMONAREA_AVG,numeric,69.872297,3181,0.0760
1,COMMONAREA_MODE,numeric,69.872297,3128,0.0744
2,COMMONAREA_MEDI,numeric,69.872297,3202,0.0761
3,NONLIVINGAPARTMENTS_AVG,numeric,69.432963,386,0.0477
4,NONLIVINGAPARTMENTS_MODE,numeric,69.432963,167,0.0463
5,NONLIVINGAPARTMENTS_MEDI,numeric,69.432963,214,0.0474
6,LIVINGAPARTMENTS_AVG,numeric,68.354953,1868,0.0926
7,LIVINGAPARTMENTS_MODE,numeric,68.354953,736,0.0979
8,LIVINGAPARTMENTS_MEDI,numeric,68.354953,1097,0.0936
9,FLOORSMIN_AVG,numeric,67.848630,305,0.1614


1. 
    * All the normalized information about the buildings contians huge number of missing values.
    * The normalized information contain:
        - APARTMENTS
        - BASEMENTAREA
        - COMMONAREA
        - ELEVATORS
        - EMERGENCYSTATE (contains only mode)
        - ENTRANCES
        - FLOORSMAX
        - FLOORSMIN
        - FONDKAPREMONT (contains only mode)
        - HOUSETYPE (contains only mode)
        - LANDAREA
        - LIVINGAPARTMENTS
        - LIVINGAREA
        - NONLIVINGAPARTMENTS
        - NONLIVINGAREA
        - TOTALAREA (contains only mode)
        - WALLSMATERIAL (contains only mode)
        - YEARS_BEGINEXPLUATATION
        - YEARS_BUILD
    * How to deal with these missing values?
2. * **OCCUPATION_TYPE** has 34.35% of missing values which can be imputed by using decision-tree based on **ORGANIZATION_TYPE**
3. * **CNT_FAM_MEMBERS** can be dealt with using **CNT_CHILDREN**.

#### Realtion between OCCUPATION_TYPE & ORGANIZATION_TYPE

In [ ]:
from scipy.stats import chi2_contingency

# Create a contingency table using the crosstab function
contingency_table = pd.crosstab(application['ORGANIZATION_TYPE'], application['OCCUPATION_TYPE'])

# Calculate the chi-squared statistic and p-value
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Calculate Cramér's V
n = len(application)
v = np.sqrt(chi2 / (n * (min(contingency_table.shape) - 1)))

print("Cramer's V: ", v)
print("p-value: ", p)

Cramer's V:  0.24968004480888786
p-value:  0.0


Since, the p-value is less than 0.05, we can conclude that there is a significant association between the two categorical variables.

#### FLAG_OWN_CAR & OWN_CAR_AGE<br>
- The Variable **OWN_CAR_AGE** has approximately 66% missing values, but that value is related to **FLAG_OWN_CAR**.
- This is because, if the consumer doesn't owns a car then the there will be no age associated with it.
- The rest of the null values(5) can be imputed using various techniques based on data where consumers already have car.
- **How to deal with null values if the person has no car?**

In [ ]:
print(f"The number of observations where a person has a car and OWN_CAR_AGE is null is {application[(application.FLAG_OWN_CAR == 'Y') & (application.OWN_CAR_AGE.isna())].shape[0]}")
print(f"The number of observations where a person has a car is {application[(application.FLAG_OWN_CAR == 'Y')].shape[0]}")

The number of observations where a person has a car and OWN_CAR_AGE is null is 5
The number of observations where a person has a car is 104587


In [ ]:
print(f"The number of observations where a person has no car and OWN_CAR_AGE is null is {application[(application.FLAG_OWN_CAR == 'N') & (application.OWN_CAR_AGE.isna())].shape[0]}")
print(f"The number of observations where a person has no car is {application[(application.FLAG_OWN_CAR == 'N')].shape[0]}")

The number of observations where a person has no car and OWN_CAR_AGE is null is 202924
The number of observations where a person has no car is 202924


### bureau.csv
* All client's previous credits provided by other financial institutions that were reported to Credit Bureau (for clients who have a loan in our sample).
* For every loan in our sample, there are as many rows as number of credits the client had in Credit Bureau before the application date.

**Notes:**
- **SK_ID_BUREAU** refers to various loans a person has taken from differnt financial institutions.
- **CREDIT_ACTIVE** has four types: Closed, Active, Sold & Bad Debt.

In [ ]:
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [ ]:
univariate_analysis(bureau)

,column_name,var_type,percent_missing,num_unique,std
0,AMT_ANNUITY,numeric,71.473490,40321,325826.9491
1,AMT_CREDIT_MAX_OVERDUE,numeric,65.513264,68251,206031.6062
2,DAYS_ENDDATE_FACT,numeric,36.916958,2917,714.0106
3,AMT_CREDIT_SUM_LIMIT,numeric,34.477415,51726,45032.0315
4,AMT_CREDIT_SUM_DEBT,numeric,15.011932,226537,677401.1310
5,DAYS_CREDIT_ENDDATE,numeric,6.149573,14096,4994.2198
6,AMT_CREDIT_SUM,numeric,0.000757,236708,1149811.3440
7,SK_ID_CURR,numeric,0.000000,305811,102938.5581
8,SK_ID_BUREAU,numeric,0.000000,1716428,532265.7286
9,DAYS_CREDIT,numeric,0.000000,2923,795.1649


In [ ]:
bureau.shape

(1716428, 17)

In [ ]:
bureau.SK_ID_CURR.value_counts()[bureau.SK_ID_CURR == 215354]

225158    9
225160    1
225157    1
225159    1
Name: SK_ID_CURR, dtype: int64

**Notes:**
- **AMT_ANNUITY**(0) is the column with most number of missing observations.
- ***THEORY:*** The observations where **AMT_ANNUITY**(0) is missing, there are no observations for them in **bureau_bal** dataframe.**(Check It!!!)**

#### AMT_ANNUITY

In [ ]:
bureau[pd.isnull(bureau["AMT_ANNUITY"])].head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [ ]:
a = bureau[pd.isnull(bureau["AMT_ANNUITY"])]
a.notnull().sum() * 100 / len(a)

SK_ID_CURR                100.000000
SK_ID_BUREAU              100.000000
CREDIT_ACTIVE             100.000000
CREDIT_CURRENCY           100.000000
DAYS_CREDIT               100.000000
CREDIT_DAY_OVERDUE        100.000000
DAYS_CREDIT_ENDDATE        94.359756
DAYS_ENDDATE_FACT          64.706050
AMT_CREDIT_MAX_OVERDUE     38.668689
CNT_CREDIT_PROLONG        100.000000
AMT_CREDIT_SUM             99.999185
AMT_CREDIT_SUM_DEBT        85.109770
AMT_CREDIT_SUM_LIMIT       67.106378
AMT_CREDIT_SUM_OVERDUE    100.000000
CREDIT_TYPE               100.000000
DAYS_CREDIT_UPDATE        100.000000
AMT_ANNUITY                 0.000000
dtype: float64

In [ ]:
bureau[pd.notnull(bureau["AMT_ANNUITY"])].head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
768,380361,5715448,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.0,0.0,67500.0,0.0,Credit card,-183,0.0
769,380361,5715449,Active,currency 1,-357,0,1119.0,NaN,NaN,0,45000.0,0.0,45000.0,0.0,Credit card,-130,2691.0
770,380361,5715451,Closed,currency 1,-917,0,-187.0,-759.0,NaN,0,74439.0,0.0,0.0,0.0,Consumer credit,-748,0.0
771,380361,5715452,Closed,currency 1,-993,0,31039.0,-831.0,NaN,0,315000.0,0.0,0.0,0.0,Credit card,-818,0.0
772,380361,5715453,Closed,currency 1,-1146,0,681.0,-780.0,NaN,0,2025000.0,0.0,0.0,0.0,Consumer credit,-769,0.0


#### AMT_CREDIT_MAX_OVERDUE

In [ ]:
bureau[pd.isnull(bureau.AMT_CREDIT_MAX_OVERDUE)].shape

(1124488, 17)

In [ ]:
bureau[(pd.isnull(bureau.AMT_CREDIT_MAX_OVERDUE)) & (pd.isnull(bureau.AMT_ANNUITY))].shape

(752407, 17)

#### CREDIT_ACTIVE, DAYS_CREDIT, CREDIT_DAY_OVERDUE, DAYS_CREDIT_ENDDATE & DAYS_ENDDATE_FACT
- **CREDIT_ACTIVE**(14) has 4 unique values: Closed, Active, Sold, Bad debt
- **DAYS_CREDIT**(9) gives us the number of days before current application did client apply for Credit Bureau credit.
- **CREDIT_DAY_OVERDUE**(10) gives us the number of days past due on CB credit at the time of application for related loan in our sample.
- **DAYS_CREDIT_ENDDATE**(5) gives us the remaining duration of CB credit (in days) at the time of application in Home Credit
- **DAYS_ENDDATE_FACT**(2) gives us the number of days since CB credit ended at the time of application in Home Credit (only for closed credit)

In [ ]:
bureau_null = bureau[pd.isnull(bureau.DAYS_CREDIT_ENDDATE)]

In [ ]:
bureau_null.apply(pd.value_counts)

### bureau_balance.csv
- Monthly balances of previous credits in Credit Bureau.
- This table has one row for each month of history of every previous credit reported to Credit Bureau – i.e the table has (#loans in sample * # of relative previous credits * # of months where we have some history observable for the previous credits) rows.

In [ ]:
bureau_bal.head()

In [ ]:
bureau_bal.shape

(27299925, 3)

In [ ]:
univariate_analysis(bureau_bal)

,column_name,var_type,percent_missing,num_unique,std
0,SK_ID_BUREAU,numeric,0.0,817395,492348.8569
1,MONTHS_BALANCE,numeric,0.0,97,23.8645
2,STATUS,categorical,0.0,8,None


### credit_card_balance.csv
- Monthly balance snapshots of previous credit cards that the applicant has with Home Credit.
- This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credit cards * # of months where we have some history observable for the previous credit card) rows.

In [ ]:
credit_bal.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.000,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.080,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,2250.0,26926.425,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,11925.0,224949.285,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,27000.0,443044.395,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [ ]:
univariate_analysis(credit_bal)

,column_name,var_type,percent_missing,num_unique,std
0,AMT_PAYMENT_CURRENT,numeric,19.998063,163209,36078.0850
1,AMT_DRAWINGS_ATM_CURRENT,numeric,19.524872,2267,28225.6886
2,AMT_DRAWINGS_OTHER_CURRENT,numeric,19.524872,1832,8201.9893
3,AMT_DRAWINGS_POS_CURRENT,numeric,19.524872,168748,20796.8870
4,CNT_DRAWINGS_ATM_CURRENT,numeric,19.524872,44,1.1004
5,CNT_DRAWINGS_OTHER_CURRENT,numeric,19.524872,11,0.0826
6,CNT_DRAWINGS_POS_CURRENT,numeric,19.524872,133,3.2406
7,AMT_INST_MIN_REGULARITY,numeric,7.948208,312266,5600.1541
8,CNT_INSTALMENT_MATURE_CUM,numeric,7.948208,121,20.0515
9,SK_ID_PREV,numeric,0.000000,104307,536469.4706


### installments_payment.csv
- Repayment history for the previously disbursed credits in Home Credit related to the loans in our sample.
- There is a) one row for every payment that was made plus b) one row each for missed payment.
- One row is equivalent to one payment of one installment OR one installment corresponding to one payment of one previous Home Credit credit related to loans in our sample.

In [ ]:
installments_payment.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [ ]:
univariate_analysis(installments_payment)

,column_name,var_type,percent_missing,num_unique,std
0,DAYS_ENTRY_PAYMENT,numeric,0.021352,3039,800.5859
1,AMT_PAYMENT,numeric,0.021352,944235,54735.7840
2,SK_ID_PREV,numeric,0.000000,997752,536202.9055
3,SK_ID_CURR,numeric,0.000000,339587,102718.3104
4,NUM_INSTALMENT_VERSION,numeric,0.000000,65,1.0352
5,NUM_INSTALMENT_NUMBER,numeric,0.000000,277,26.6641
6,DAYS_INSTALMENT,numeric,0.000000,2922,800.9463
7,AMT_INSTALMENT,numeric,0.000000,902539,50570.2544


### POS_CASH_balance.csv
- Monthly balance snapshots of previous POS (point of sales) and cash loans that the applicant had with Home Credit.
- This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credits * # of months in which we have some history observable for the previous credits) rows.

In [ ]:
pos_cash_bal.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [ ]:
univariate_analysis(pos_cash_bal)

,column_name,var_type,percent_missing,num_unique,std
0,CNT_INSTALMENT_FUTURE,numeric,0.260835,79,11.1091
1,CNT_INSTALMENT,numeric,0.260675,73,11.9951
2,SK_ID_PREV,numeric,0.000000,936325,535846.5307
3,SK_ID_CURR,numeric,0.000000,337252,102763.7451
4,MONTHS_BALANCE,numeric,0.000000,96,26.0666
5,SK_DPD,numeric,0.000000,3400,132.7140
6,SK_DPD_DEF,numeric,0.000000,2307,32.7625
7,NAME_CONTRACT_STATUS,categorical,0.000000,9,None


In [ ]:
pos_cash_bal.shape

(10001358, 8)

### previous_application.csv
- All previous applications for Home Credit loans of clients who have loans in our sample.
- There is one row for each previous application related to loans in our data sample.

In [ ]:
prev_application.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
prev_application[prev_application.SK_ID_CURR == 271877]

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.000000,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
735737,1696966,271877,Consumer loans,68258.655,1800000.0,1754721.0,180000.0,1800000.0,SATURDAY,18,Y,1,0.101325,NaN,NaN,XAP,Refused,-472,Cash through the bank,SCO,NaN,Repeater,Clothing and Accessories,POS,XNA,Regional / Local,55,Furniture,36.0,low_normal,POS industry with interest,NaN,NaN,NaN,NaN,NaN,NaN
1567518,2154916,271877,Consumer loans,12417.390,108400.5,119848.5,0.0,108400.5,SUNDAY,14,Y,1,0.000000,NaN,NaN,XAP,Approved,-548,Cash through the bank,XAP,NaN,New,Furniture,POS,XNA,Stone,196,Furniture,12.0,middle,POS industry with interest,365243.0,-512.0,-182.0,-392.0,-387.0,0.0


In [ ]:
univariate_analysis(prev_application)

,column_name,var_type,percent_missing,num_unique,std
0,RATE_INTEREST_PRIMARY,numeric,99.643698,148,0.0877
1,RATE_INTEREST_PRIVILEGED,numeric,99.643698,25,0.1009
2,AMT_DOWN_PAYMENT,numeric,53.636480,29278,20921.4954
3,RATE_DOWN_PAYMENT,numeric,53.636480,207033,0.1078
4,DAYS_FIRST_DRAWING,numeric,40.298129,2838,88916.1158
5,DAYS_FIRST_DUE,numeric,40.298129,2892,72444.8697
6,DAYS_LAST_DUE_1ST_VERSION,numeric,40.298129,4605,106857.0348
7,DAYS_LAST_DUE,numeric,40.298129,2873,149647.4151
8,DAYS_TERMINATION,numeric,40.298129,2830,153303.5167
9,NFLAG_INSURED_ON_APPROVAL,numeric,40.298129,2,0.4711


**Notes:**
- **RATE_INTEREST_PRIMARY**(0) & **RATE_INTEREST_PRIVILEGED**(1) are the two columns with 99.6436% of null observation, which is not enought for analysis, so we will be droping them.
- **NFLAG_INSURED_ON_APPROVAL**(9) is a categorical variable but is being treated as numeric.**(Change it!!!)**
- **NFLAG_LAST_APPL_IN_DAY**(18) is a categorical variable but is being treated as numeric.**(Change it!!!)**

In [ ]:
prev_application.shape

(1670214, 37)

#### NAME_TYPE_SUITE

In [ ]:
prev_application.NAME_TYPE_SUITE.value_counts()

Unaccompanied      508970
Family             213263
Spouse, partner     67069
Children            31566
Other_B             17624
Other_A              9077
Group of people      2240
Name: NAME_TYPE_SUITE, dtype: int64

In [ ]:
prev_application.NAME_TYPE_SUITE.isnull().sum()

820405

#### PRODUCT_COMBINATION & NAME_CONTRACT_TYPE


In [ ]:
prev_application.PRODUCT_COMBINATION.value_counts()

Cash                              285990
POS household with interest       263622
POS mobile with interest          220670
Cash X-Sell: middle               143883
Cash X-Sell: low                  130248
Card Street                       112582
POS industry with interest         98833
POS household without interest     82908
Card X-Sell                        80582
Cash Street: high                  59639
Cash X-Sell: high                  59301
Cash Street: middle                34658
Cash Street: low                   33834
POS mobile without interest        24082
POS other with interest            23879
POS industry without interest      12602
POS others without interest         2555
Name: PRODUCT_COMBINATION, dtype: int64

In [ ]:
prev_application['PRODUCT_COMBINATION'].isnull().sum()

346

In [ ]:
prev_application[prev_application.NAME_CONTRACT_TYPE == 'XNA'].shape[0]

346

### Merging Bureau & Bureau_Balance

In [ ]:
bureau_merged = pd.merge(left=bureau,
                        right=bureau_bal,
                        how='inner',
                        on='SK_ID_BUREAU')

In [ ]:
print(f'No. of observations in bureau: {bureau.shape[0]}')
print(f'No. of observations in bureau_balance: {bureau_bal.shape[0]}')
print(f'No. of observations in bureau_merged: {bureau_merged.shape[0]}')

There are a few reasons why the merged dataframe resulting from a left join could have a smaller shape than the right dataframe ('**bureau_bal**' in this case).

1. The left dataframe ('**bureau**') may not have any matching rows in the right dataframe ('**bureau_bal**'). In this case, the merged dataframe will have a row for each row in the left dataframe, but with NaN values in the columns from the right dataframe for the rows that don't match.

2. The left dataframe ('**bureau**') may have multiple rows that match a single row in the right dataframe ('**bureau_bal**'). In this case, the merged dataframe will have a single row for each row in the left dataframe, with the columns from the right dataframe repeated for each match.